# Трансформери и језички задаци

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Petlja/specit4_ai_radni/blob/main/10-transformeri_i_jezicki_zadaci.ipynb)

У овој свесци можеш да опробаш како се користе трансформери у задацима обраде природних језика као што су анализа сентимената и генерисање сажетака. Примери ће бити везани за енглески језик јер се библиотеке и функциналности понајпре прилагођавају овом језику.

На самом почетку учитаћемо стандардне библиотеке које су нам неопходне за даљи рад.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

За задатке које планирамо да покријемо биће нам потребна библиотека `transformers` која обједињује различите врсте трансформера и алате који омогућавају удобнији рад са њима. Да би ова библиотека могла да се користи у окружењу Google Colab потребно ју је инсталирати наредбом `!pip install transformers` а потом и учитати наредбом `include transformers`. Ми ћемо примарно користити функцију `pipeline` ове библиотеке али ћемо продискутовати још неке њене могућности.

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00


In [ ]:
import transformers

## Задатак анализе сентимената

Задатак анализе сентимената (енгл. sentiment analysis) је задатак  препознавања емоција или ставова присутних у неком тексту. Само препзнавање је доста базичније у односу на начин на који то људи умеју али има своју важну улогу у разумевању кориснички генерисаних садржаја попут коментара или прегледа. Најчеше се сусрећемо са задатком препознавања позитивних и негативних садржаја где позитивни садржаји означавају нешто похвално и лепо а негативни садржаји критике и замерке. Самом задатку анализе сентимената из угла машинског учења приступамо као задатку бинарне класификације. Након што припремимо адекватно репрезентације текстуалних улаза, можемо применити било који алгоритам класификације.

Следећи блок кода ће нам омогућити да креирамо функционалност `analiziraj_sentiment` која за нас обједињује кораке креирања репрезентације текста а потом и покретања већ обученог класификатора за анализу сентимената. За њено креирање искористићемо функцију `pipeline` и посебно аргументом `task` нагласити да желимо да се бавимо анализом сентимената.

In [ ]:
analiziraj_sentiment = transformers.pipeline(task='sentiment-analysis')


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Учитаној функционалности можемо да задајемо улазе за које желимо да добијемо оцену сентимента. Као излаз ћемо добити име класе `POSITIVE` или `NEGATIVE`, као и вредност `score` у интервалу од 0 до 1 која указује колико је модел класификације сигуран у своју одлуку.

Следи неколико примера.

In [ ]:
analiziraj_sentiment("We are very excited to learn more on sentiment analysis!")

[{'label': 'POSITIVE', 'score': 0.9994511008262634}]

In [ ]:
analiziraj_sentiment("We didn't like the food. It was too salty.")

[{'label': 'NEGATIVE', 'score': 0.9992080330848694}]

In [ ]:
analiziraj_sentiment("The movie was super interesting, but the end was quite boring.")

[{'label': 'NEGATIVE', 'score': 0.9983773231506348}]

Док је емоција узхићења тј. недопадања била прилично јасно изражена у првим двема реченицама које смо тестирали, у трећој реченици имамо занимљиву мешавину. Да ли би се сложио са оценом коју је дао класификатор?

Можеш да наставиш и даље да тестираш ову функционалност тако што ћеш проверити како на одлуке класификатора утичу придеви попут *amazing*, *wonderful*, *boring*, *annoying* и њихова комбинација. Можеш да провериш и како се класификатор понаша када је у реченици присутна негација, на пример, када кажеш да нешто *није сјајно*.

## Задатак сумаризације

Користећи принцип сличан претходном можемо да се опробамо и у задатку сумаризације. За задати текст задатак сумаризације (енгл. summarization) подразумева генерисање сажетака, краће текстуалне форме која садрже важне и релевантне информације полазног текста. С обзиром да очекивања од сажетака могу да варирају од корисника до корисника (доживљај важног и релевантног зависи од пуно фактора) за задати текст је могуће креирати више различитих сажетака.

Следећи блок кода ће нам омогућити да креирамо функционалност `generisi_sazetak` која за нас обједињује кораке креирања репрезентације задатог текста а потом и покретања већ обученог модела за генерисање сажетака. Користићемо, такође, функцију `pipeline` и посебно аргументом `task` нагласити да желимо да се бавимо сумаризацијом.

In [ ]:
generisi_sazetak = transformers.pipeline(task='summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


За тестирање учитане функционалности, искористићемо један пасус чланка о Николи Тесли преузет са Википедије.

In [ ]:
input_text = """
In 1884, Edison manager Charles Batchelor, who had been overseeing the Paris installation,
was brought back to the United States to manage the Edison Machine Works,
a manufacturing division situated in New York City, and asked that Tesla be brought to the United States as well.
In June 1884, Tesla emigrated and began working almost immediately at the Machine Works on Manhattan's Lower East Side,
an overcrowded shop with a workforce of several hundred machinists, laborers, managing staff, and 20 "field engineers"
struggling with the task of building the large electric utility in that city. As in Paris, Tesla was working on
troubleshooting installations and improving generators. Historian W. Bernard Carlson notes Tesla may have met
company founder Thomas Edison only a couple of times. One of those times was noted in Tesla's autobiography where,
after staying up all night repairing the damaged dynamos on the ocean liner SS Oregon, he ran into Batchelor and Edison,
who made a quip about their "Parisian" being out all night. After Tesla told them he had been up all night fixing the Oregon,
Edison commented to Batchelor that "this is a damned good man". One of the projects given to Tesla was to develop an
arc lamp-based street lighting system. Arc lighting was the most popular type of street lighting but it required
high voltages and was incompatible with the Edison low-voltage incandescent system, causing the company to lose
contracts in some cities. Tesla's designs were never put into production, possibly because of technical improvements
in incandescent street lighting or because of an installation deal that Edison made with an arc lighting company.
"""

Приликом позива функционалности генерисaћемо тачно један сажетак чију ћемо дужину ограничити на максималних 150 ”речи”. Сам садржај сажетка ћемо моћи да прочитамо уз помоћ својства `summary_text`.

In [ ]:
sazetak = generisi_sazetak(input_text, max_length=150)

In [ ]:
sazetak[0]['summary_text']

" Tesla emigrated to the United States in 1884 to work at the Edison Machine Works on Manhattan's Lower East Side . Tesla's designs were never put into production, possibly because of technical improvements in incandescent street lighting or because of an installation deal with an arc lighting company . Tesla may have met Thomas Edison only a couple of times ."

Опробај се још са неким улазима и подешавањима дужине сажетка. У наставку ћемо више говорити и о моделима који се користе па можеш да искористиш и неки други модел.

## Репрезентација текста

Обе претходне функционалности су за нас припремале одговарајуће текстуалне репрезентације и позивале одговарајуће моделе. Ако се вратимо и погледамо поруке које смо добили приликом позивања функције `pipeline` приметићемо да је за задатак анализе сентимената коришћен модел `distilbert-base-uncased-finetuned-sst-2-english` док је за задатак сумаризације коришћен модел `sshleifer/distilbart-cnn-12-6`. Ова имена смо могли и експлицитно да задамо приликом навођења функције `pipeline` преко аргумента `model`. У самој библиотеци на располагању су још неки модели за ове задатке.

Пракса је да сваки модел буде упарен са својим токeнизатором. Токенизатор је алат који ће се постарати да улаз буде у форми коју модел разуме. Да бисмо демонстрирали како ово функционише и сазнали нешто више о начину прпреме текста, учитаћемо директно модел `distilbert-base-uncased-finetuned-sst-2-english`. За учитавање модела искористићемо функцију `AutoModelForSequenceClassification`, док ћемо за учитавање токенизатора искористити функцију `AutoTokenizer`.

In [ ]:
ime_modela = 'distilbert-base-uncased-finetuned-sst-2-english'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(ime_modela)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(ime_modela)

Трансформер који смо одабрали користи токенизацију парчићима речи (енгл. subword tokenization). Парчићи речи се пажљиво издвајају проласком кроз неку велику количину текста и бирају тако да се њиховим комбиновањем може реконструисати највећи део текста. Ово се не ради ручно, већ се позива сам алгоритам за креирање парчића. Сви овако издвојени парчићи (зовемо их и токенима) представљају вокабулар. Сада ћемо видети колико токена се налази у вокабулару овог трансформера, а потом и исписати неке од њих. Приликом исписа ћеш видети да неки токени почињу карактерима ## - они означавају да то нису целе речи већ парчићи који се комбинују са другим речима. Ово нам даје могућност да представимо и речи које су присутне у тексту а које нису биле укључене у процес одабира и генерисања парчића речи.

Да бисмо одредили број токена вокабулара користићемо својство `tokenizer.vocab_size`.

In [ ]:
print('Број токена у вокабулару: ', tokenizer.vocab_size)

Број токена у вокабулару:  30522


Да бисмо издвојили неке токене вокабулара користићемо својство `tokenizer.vocab`.

In [ ]:
broj_tokena_za_prikaz = 50
vokabular = {id: token for token, id in tokenizer.vocab.items()}

for indeks, (id, token) in enumerate(vokabular.items()):
  token = vokabular[id]
  print("ID: {id} \t token: {token}".format(id=id, token=token))

  if indeks == broj_tokena_za_prikaz:
    break

ID: 3021 	 token: bill
ID: 17133 	 token: lively
ID: 21066 	 token: bicycles
ID: 3829 	 token: kitchen
ID: 9560 	 token: outcome
ID: 11451 	 token: ##kers
ID: 17608 	 token: tsar
ID: 8970 	 token: introduce
ID: 10019 	 token: zones
ID: 23909 	 token: ##ocation
ID: 27786 	 token: slacks
ID: 2641 	 token: considered
ID: 15527 	 token: mutant
ID: 4208 	 token: focused
ID: 11737 	 token: dim
ID: 23747 	 token: anatolia
ID: 11687 	 token: pad
ID: 15894 	 token: torque
ID: 19307 	 token: ##sible
ID: 3669 	 token: ##li
ID: 23839 	 token: timetable
ID: 11054 	 token: whispers
ID: 7746 	 token: divine
ID: 17443 	 token: hoover
ID: 24031 	 token: yamaha
ID: 18732 	 token: unison
ID: 17876 	 token: elongated
ID: 11275 	 token: baltic
ID: 3565 	 token: super
ID: 8868 	 token: coached
ID: 14434 	 token: moody
ID: 1102 	 token: đ
ID: 21601 	 token: jammed
ID: 22981 	 token: postage
ID: 13497 	 token: patrons
ID: 18169 	 token: laguna
ID: 22723 	 token: ##tica
ID: 26378 	 token: ##hri
ID: 19807 	 tok

Токени у вокабулару имају своје јединствене идентификаторе. Њих смо искористили и за дохватање одређеног броја токена из вокабулара. То даље значи да сваки улаз прво треба записати комбиновањем парчића речи, а затим сваком парчићу речи придружити одговарајући идентификатор. На овај начин долазимо и до нумеричке репрезентације текста неопходнe за даљу примену модела.

За припрему репрезентација нам у раду са библиотеком `transformers` помаже сам токенизатор.

In [ ]:
ulaz = "We are excited to learn about transformers and natural language processing."

In [ ]:
tokenizer(ulaz)

{'input_ids': [101, 2057, 2024, 7568, 2000, 4553, 2055, 19081, 1998, 3019, 2653, 6364, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Излаз функције су, као што можемо да видимо, идентификатори појединачних токена представљени низом `input_ids` и низ `attention_mask` који помаже у функционалностима механизма пажње. На почетку низа токена се увек додаје и један посебан почетни токен чији је идентификатор 101 (такозвани стартни CLS токен) и један крајњи токен чији је идентификатор 102 (такозвани SEP токен).  

Да бисмо могли да видимо како мрежа обрађује овај улаз припремићемо пакетић са овим улазом. У припреми самог пакета ћемо користити и нека додатна подешавања која ће улаз или проширити нулама тако да испуни максималну дужину од 512 токена или по потреби скратити на ову дужину (зато је наведен атрибут `padding` са вредношћу `True`,  атрибут `max_length` са вредношћу 512 и атрибут `trancation` са вредношћу `True`).

In [ ]:
ulaz_paketic = tokenizer([ulaz], padding=True, truncation=True, max_length=512, return_tensors="pt")

Овако припремљени улаз даље можемо проследити моделу.

In [ ]:
rezultat = model(**ulaz_paketic)

Прво ћемо исписати сам резултат, а потом и искористити функционалност која ће за нас израчунати неопходну вредност оцене.

In [ ]:
rezultat

SequenceClassifierOutput(loss=None, logits=tensor([[-4.0110,  4.2654]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
from torch import nn
nn.functional.softmax(rezultat.logits, dim=-1)

tensor([[2.5438e-04, 9.9975e-01]], grad_fn=<SoftmaxBackward0>)

Прва израчуната вредност одговара оцени негативног сентимента, а друга оцени позитивног сентимента. Како је она већа, закључак модела би био да је реч о позитивном садржају. У то се можемо уверити и позивом функционалности `analiziraj_sentiment`.

In [ ]:
analiziraj_sentiment(ulaz)

[{'label': 'POSITIVE', 'score': 0.9997456669807434}]

Библиотека `transformers` нуди могућност рада и са многим другим моделима и функционалностима попут конверзије говора у текст, одговора на питања, обележавања ентитета у тексту и многих других. Можда ће ти бити занимљиво да истражиш. Више о томе можеш пронаћи на званичној страници библиотеке https://huggingface.co/docs/transformers/index. Библиотеку развија и одржава заједница HuggingFace 🤗.